In [1]:
#ENCODER DECODER MODEL FOR WORD LEVEL EMBEDDING

In [1]:
## LOADING THE REQUIRED LIBRARIES
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import tensorflow as tf
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="2,3,4,6,7"

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib

Num GPUs Available:  5


In [4]:
## LOADING THE PROCESSED DATASET  

df= pd.read_csv('DATA/etoori_train.csv')
df["dec_output"] = df.dec_input
df.head(10)


,enc_input,dec_input,dec_output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप ...,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपन...,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपन...
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल ...,देश में हिन्दी को विस्थापित करने का षड़यंत्र च...,देश में हिन्दी को विस्थापित करने का षड़यंत्र च...
2,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...
3,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...
4,तब तक के लिए हमें विराम ले की अनुमति दीजिए ।,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।
5,देव स्थेन तक सड़क बनाने की परियोजना चल रही है ।,देव स्थान तक सड़क बनाने की परियोजना चल रही है ।,देव स्थान तक सड़क बनाने की परियोजना चल रही है ।
6,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह आरएनएस ...,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह ने आरएन...,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह ने आरएन...
7,दो दिन पहले ज्ञानजी पोस्ट ठेली जिसमें,दो दिन पहले ज्ञानजी ने पोस्ट ठेली जिसमें,दो दिन पहले ज्ञानजी ने पोस्ट ठेली जिसमें
8,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी ज...,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी ज...,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी ज...
9,देर से जाऊंगा तो दुकां बंद हो जाएंगी ।,देर से जाऊंगा तो दुकानें बंद हो जाएंगी ।,देर से जाऊंगा तो दुकानें बंद हो जाएंगी ।


In [5]:
# Adding start and end token
## THE INPUTS TO THE DECODER REQUIRES SPECIAL TOKENS FOR THE START AND THE END SO WE ARE GOING TO USE 
## <start> AS BEGINING TOKEN
## <end>  AS END TOKEN

df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>" 
df

,enc_input,dec_input,dec_output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप ...,<start> परन्तु वे दोनों उन बातों को ज़्यादा सम...,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपन...
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल ...,<start> देश में हिन्दी को विस्थापित करने का षड...,देश में हिन्दी को विस्थापित करने का षड़यंत्र च...
2,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...,<start> तीन साल पहले कातिलाना हमले के प्रकरण म...,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर ...
3,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...,<start> रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवें...,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम स...
4,तब तक के लिए हमें विराम ले की अनुमति दीजिए ।,<start> तब तक के लिए हमें विराम लेने की अनुमति...,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।...
...,...,...,...
139995,लेकिन फिर भी फिल्म को लोगों जित निगेटिव कमेंट्...,<start> लेकिन फिर भी फिल्म को लोगों ने जितने न...,लेकिन फिर भी फिल्म को लोगों ने जितने निगेटिव क...
139996,बेटी से छेड़छाड़ के साथ वह कई बार दुराचार की क...,<start> बेटी से छेड़छाड़ के साथ उसने कई बार दु...,बेटी से छेड़छाड़ के साथ उसने कई बार दुराचार की...
139997,आज यह कलाकार बालीवुड की चकाचौंध से दूर अपने गा...,<start> आज यह कलाकार बालीवुड की चकाचौंध से दूर...,आज यह कलाकार बालीवुड की चकाचौंध से दूर अपने गा...
139998,यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओं में सफ...,<start> यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओ...,यदि ये इस वर्ष अपनी प्रतियोगी परीक्षाओं में सफ...


In [6]:
# Splitting And Sampling around 100k datapoints
#THE TOTAL DATASET HAS 500K DATAPOINTS WHICH WILL TAKE MUCH HIGHER TRAINING TIME. THEREFORE I AM SAMPLING ONE-FIFTH OF THE TOTAL DATASET

#df_sampled = pd.concat((df[df.enc_input].sample(frac= 0.2,random_state=1)))
#df_sampled = df.sample(frac = 0.1)
print(df.shape)
#print(df_sampled.shape)

(140000, 3)


In [7]:
## ONCE THE DATA IS SAMPLED WE ARE SPLITTIND THE DATA IN TO TRAIN AND TEST

df_train ,df_val = train_test_split(df,test_size=0.2,random_state = 3)


In [8]:
## IN THE COLUMN WHICH HAS DECODER INPUTS ADDING "<end>" TOKEN TO BE LEARNED BY THE TOKENIZER

df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"
df_train


In [9]:
## VALIDATION DATA
df_val

,enc_input,dec_input,dec_output
90522,महुआ का तेल जलाने के काम में लाया जाता हैं ।,<start> महुआ का तेल जलाने के काम में लाया जाता...,महुआ का तेल जलाने के काम में लाया जाता है । <...
133504,आजकल ट्रेन में एक से ज़्यादा टीटी भी होते हैंं...,<start> आजकल ट्रेन में एक से ज़्यादा टीटी भी ह...,आजकल ट्रेन में एक से ज़्यादा टीटी भी होते हैं ...
2373,शाम के वक्त बारी बारी से घर के सभी सदस्य रश्मि...,<start> शाम के वक्त बारी बारी से घर के सभी सदस...,शाम के वक्त बारी बारी से घर के सभी सदस्य रश्मि...
136466,उनमें शिवकुमार दीपक प्रमुख है ।,<start> उनमें शिवकुमार दीपक प्रमुख हैं ।,उनमें शिवकुमार दीपक प्रमुख हैं । <end>
5198,हमारे देश में संविधान द्वारा अब स्त्रियों को अ...,<start> हमारे देश में संविधान द्वारा अब स्त्रि...,हमारे देश में संविधान द्वारा अब स्त्रियों को अ...
...,...,...,...
111735,सरकार किसी पुल या सड़क का लोकार्पण करती हैं तो...,<start> सरकार किसी पुल या सड़क का लोकार्पण करत...,सरकार किसी पुल या सड़क का लोकार्पण करती है तो ...
19247,उन्होंने हमें सिखाया कि कैसे संघर्ष किया जाता ...,<start> उन्होंने हमें सिखाया कि कैसे संघर्ष कि...,उन्होंने हमें सिखाया कि कैसे संघर्ष किया जाता ...
57036,लेकिन यह इतिहास का अन्त नहीं हैं ।,<start> लेकिन यह इतिहास का अन्त नहीं है ।,लेकिन यह इतिहास का अन्त नहीं है । <end>
73689,रतन टाटा मल्टीब्रांड रिटेल में एफडीआई का स्वाग...,<start> रतन टाटा ने मल्टीब्रांड रिटेल में एफडी...,रतन टाटा ने मल्टीब्रांड रिटेल में एफडीआई का स्...


In [10]:
## HERE I AM SAMPLING 1000 POINTS FROM THE DATAFRAME AS TEST DATA WHICH ARE NOT PRESEENT IN THE TRAIN AND VALIDAION DATA
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values]),1000,replace= False,)]
df_test

,enc_input,dec_input,dec_output
80240,हमें आने वाले हफ्तों में रोवियो मोबाइल द्वारा ...,<start> हमें आने वाले हफ्तों में रोवियो मोबाइल...,हमें आने वाले हफ्तों में रोवियो मोबाइल द्वारा ...
32433,आज कमेन्ट मोदेरेशन सक्षम हैंं ।,<start> आज कमेन्ट मोदेरेशन सक्षम हैं ।,आज कमेन्ट मोदेरेशन सक्षम हैं । <end>
135530,राजस् थेन स्कूल टीचर्स यूनियन के अध्यक्ष आर पी...,<start> राजस् थान स्कूल टीचर्स यूनियन के अध्यक...,राजस् थान स्कूल टीचर्स यूनियन के अध्यक्ष आर पी...
3354,सेक्स का कारक ग्रह भी शुक्र ही हैं तथा शुक्र ए...,<start> सेक्स का कारक ग्रह भी शुक्र ही है तथा ...,सेक्स का कारक ग्रह भी शुक्र ही है तथा शुक्र एक...
125119,वह एक गिलास शर्बत,<start> उसने एक गिलास शर्बत,उसने एक गिलास शर्बत <end>
...,...,...,...
123821,बीकॉम ऑनर्स और इको ऑनर्स कोर्स स्टूडेंट की पसं...,<start> बीकॉम ऑनर्स और इको ऑनर्स कोर्स स्टूडें...,बीकॉम ऑनर्स और इको ऑनर्स कोर्स स्टूडेंट की पसं...
40525,मरने वालों में बस में सवार यात्री थी ।,<start> मरने वालों में बस में सवार यात्री थे ।,मरने वालों में बस में सवार यात्री थे । <end>
72286,यान्त्रिक रूप से क्षमा मांगना सम्पर्क बढ़ा का ...,<start> यान्त्रिक रूप से क्षमा मांगना सम्पर्क ...,यान्त्रिक रूप से क्षमा मांगना सम्पर्क बढ़ाने क...
90240,मैंनेने इस शहर में खुद का स् वागत बांहें खोलकर...,<start> मैंने इस शहर में खुद का स् वागत बांहें...,मैंने इस शहर में खुद का स् वागत बांहें खोलकर न...


In [11]:
# Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
## TOKENIZER FOR ENCODER INPUT
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))

In [13]:
# TOKENIZER FOR DECODER INPUT
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

In [14]:
## THIS CLASS CONVERTS TEXT DATA TO INTEGER SEQUENCES AND RETURNS THE PADDED SEQUENCES

class Dataset :
    def __init__(self, data , tk_inp ,tk_out, max_len):
        ## SETTING THE REQUIRED ATTRIBUTES
        self.encoder_inp = data["enc_input"].apply(str).values
        self.decoder_inp = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp
        self.tk_out = tk_out
        self.max_len = max_len
        
    def __getitem__(self,i):
        # INPUT SEQUENCES
        self.encoder_seq = self.tk_inp.texts_to_sequences([self.encoder_inp[i]])
        # DECODER INPUT SEQUENCES 
        self.decoder_inp_seq = self.tk_out.texts_to_sequences([self.decoder_inp[i]])
        # DECODER INPUT SEQUENCES
        self.decoder_out_seq = self.tk_out.texts_to_sequences([self.decoder_out[i]])
        
        # PADDING THE ENCODER INPUT SEQUENCES
        self.encoder_seq = pad_sequences(self.encoder_seq, padding="post",maxlen = self.max_len)
        # PADDING THE DECODER INPUT SEQUENCES
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, padding="post",maxlen = self.max_len)
        # PADDING DECODER OUTPUT SEQUENCES
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq ,padding="post", maxlen = self.max_len)

        ##  RETURNING THE ENCODER INPUT , DECODER INPUT , AND DECODER OUTPUT
        return self.encoder_seq ,  self.decoder_inp_seq,  self.decoder_out_seq
    
    def __len__(self):
        # RETURN THE LEN OF INPUT ENDODER
        return len(self.encoder_inp)

In [15]:
## THIS CLASS CONVERTES THE DATASET INTO THE REQUIRED BATCH SIZE

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self,batch_size,dataset):
        # INTIALIZING THE REQUIRED VARIABLES 
        self.dataset = dataset
        self.batch_size = batch_size
        self.totl_points = self.dataset.encoder_inp.shape[0]
        
    def __getitem__(self,i):
        # STATING THE START AND STOP VATIABLE CONTAINGING INDEX VALUES FOR EACH BATCH
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        # PLACEHOLDERS FOR BATCHED DATA
        batch_enc =[]
        batch_dec_input = []
        batch_dec_out =[]

        for j in range(start,stop): 
            
            a,b,c = self.dataset[j] 
            batch_enc.append(a[0]) 
            batch_dec_input.append(b[0])
            batch_dec_out.append(c[0]) 
        
        # Conveting list to array   
        batch_enc = (np.array(batch_enc)) 
        batch_dec_input = np.array(batch_dec_input)
        batch_dec_out = np.array(batch_dec_out)
        
        ## RETURNING BATCHED DATA IN REQUIRED FORM
        return [batch_enc , batch_dec_input],batch_dec_out
    
    def __len__(self):
        # Returning the number of batches
        return int(self.totl_points/self.batch_size)

In [16]:
# FORMING OBJECTS OF DATASET AND DATALOADER FOR TRAIN DATASET
#train_dataset = Dataset(df_train,tk_inp,tk_out,35)
train_dataset = Dataset(df_train,tk_inp,tk_out,50)
train_dataloader = Dataloader( batch_size = 32, dataset=train_dataset)

# FORMING OBJECTS OF DATASET AND DATALOADER FOR VALIDATION DATASET
#val_dataset = Dataset(df_val , tk_inp,tk_out,35)
val_dataset = Dataset(df_val , tk_inp,tk_out,50)
val_dataloader = Dataloader(batch_size=32 , dataset=val_dataset)

In [17]:
# ENCODER DECODER MODEL
## LOADING THE TENSORFLOW LIBRARIES

from tensorflow.keras import layers
from tensorflow.keras import Model

In [18]:
## DEFINING THE ENCODER LAYER AS A FUNCTION

def encoder(input_shape,vocab, emb_output, lstm_units, enc_input):
    '''THIS FUNCTION TAKES IN THE SEQUENCES AND RETURNS THE ENCODER OUTPUT'''
    ## FIRST LAYER : EMBEDDING LAYER
    enc_emb = layers.Embedding(vocab, emb_output,mask_zero = True,input_length=input_shape)(enc_input)
    ## SECOND LAYER : LSTM LAYER
    enc_lstm , enc_state_h,enc_state_c = layers.LSTM(units= lstm_units,return_sequences=True,return_state=True)(enc_emb)
    ## RETURNING THE LSTM OUTPUTS AND STATES
    return enc_lstm , enc_state_h,enc_state_c


## DEFINING THE DECODER LAYER AS A FUNCTION 
def decoder(input_shape,vocab, emb_output, lstm_units,enc_states, dec_input):
  ## FIRST LAYER : EMBEDDING LAYER
  dec_emb = layers.Embedding(vocab, emb_output , mask_zero = True,input_length=input_shape)(dec_input)
  ## SECONG LAYER : LSTM LAYER
  dec_lstm, dec_state_h,dec_state_c = layers.LSTM(units=lstm_units,return_sequences=True,return_state=True)(dec_emb,initial_state= enc_states)
  ## RETURNING THE LSTM OUTPUTS AND STATES
  return dec_lstm, dec_state_h,dec_state_c

In [19]:
## DEFINING THE MODEL ARCHITECTURE

# INPUT LAYER
#enc_input = layers.Input(shape=(35))
enc_input = layers.Input(shape=(50))
# ENCODER DEFINED FORM FUNCTON ABOVE
#enc_lstm , enc_state_h,enc_state_c = encoder(35,len(tk_inp.word_index)+1 , 300 ,256, enc_input )
enc_lstm , enc_state_h,enc_state_c = encoder(50,len(tk_inp.word_index)+1 , 300 ,256, enc_input )


# DECODER INPUT LAYER
#dec_input = layers.Input(shape = (35))
dec_input = layers.Input(shape = (50))
# DECODER DEFINEA FROM ABOVE FUNCTION
#dec_lstm , dec_state_h,dec_state_c = decoder(35,len(tk_out.word_index)+1 , 300 , 256 , [enc_state_h,enc_state_c],dec_input)
dec_lstm , dec_state_h,dec_state_c = decoder(50,len(tk_out.word_index)+1 , 300 , 256 , [enc_state_h,enc_state_c],dec_input)
# DENCSE LAYER CONNECTOD TO DECODER OUTPUT
dense = layers.Dense(len(tk_out.word_index)+1,activation="softmax")(dec_lstm)

# MODEL DEFINING
model  = Model(inputs=[enc_input,dec_input],outputs=dense)

2022-10-18 21:11:47.071671: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 21:11:50.502470: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10413 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:07:00.0, compute capability: 6.1
2022-10-18 21:11:50.507042: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10413 MB memory:  -> device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-10-18 21:11:50.512798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

In [20]:
# MODEL SUMMARY
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      22148700    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 50, 300)      21891300    ['input_2[0][0]']                
                                                                                              

In [21]:
## DEFINING THE CALLBACKS
callback =[ tf.keras.callbacks.ModelCheckpoint( "models/hindi_trainable_word_emb.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001)
]

## STORING THE NUMBER OF STEPS IN ONE EPOCH FOR TRAIN AND VALIDATION DATASET
train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

# COMPILING THE MODEL
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
## FITTING THE MODEL
model.fit(train_dataloader,steps_per_epoch=train_steps,epochs=60,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback)

Epoch 1/60


2022-10-18 21:12:36.146427: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101


3500/3500 [==============================] - 417s 117ms/step - loss: 2.0455 - accuracy: 0.2638 - val_loss: 1.5739 - val_accuracy: 0.3971
Epoch 2/60
3500/3500 [==============================] - 406s 116ms/step - loss: 1.4220 - accuracy: 0.4519 - val_loss: 1.2459 - val_accuracy: 0.4976
Epoch 3/60
3500/3500 [==============================] - 405s 116ms/step - loss: 1.0934 - accuracy: 0.5389 - val_loss: 1.0683 - val_accuracy: 0.5512
Epoch 4/60
3500/3500 [==============================] - 405s 116ms/step - loss: 0.8556 - accuracy: 0.6096 - val_loss: 0.9604 - val_accuracy: 0.5884
Epoch 5/60
3500/3500 [==============================] - 405s 116ms/step - loss: 0.6728 - accuracy: 0.6731 - val_loss: 0.8967 - val_accuracy: 0.6138
Epoch 6/60
3500/3500 [==============================] - 405s 116ms/step - loss: 0.5307 - accuracy: 0.7313 - val_loss: 0.8507 - val_accuracy: 0.6354
Epoch 7/60
3500/3500 [==============================] - 404s 115ms/step - loss: 0.4235 - accuracy: 0.7798 - val_loss: 0.826

In [23]:
# LOADING THE WEIGHTS FOR BEST MODEL
#model.load_weights("model_save/word_trainable_embedding/besh.h5")
model.built = True
model.load_weights("models/hindi_trainable_word_emb.h5")
#model.load_weights("models/hindi_trainable_word_emb_small.h5")


In [44]:
## THIS FUNCTION IS USED IN THE INFERENCE TIME TO PREDICT THE RESULTS GIVEN THE INPUT TEXT

def predict(inp , model):
    ##  TAKES INPUT AS TEXT AND THE MODEL

    # CONVERT TEXT INPUT TO SEQUENCES 
    seq = tk_inp.texts_to_sequences([inp])
    # PADDING THE SEQUENCE
    #seq = pad_sequences(seq,maxlen = 35,padding="post")
    seq = pad_sequences(seq,maxlen = 50,padding="post")
    ## INITIAL STATES FOR ENCODER
    state = [tf.zeros(shape=(1,256)),tf.zeros(shape= (1,256))]

    # SEQUENCE TO EMBEDDING
    enc_emb  = model.layers[2](seq)
    # PASSING EMBBEDDED SEQUENCES TO LSTM LAYER
    enc_output,state_h,state_c= model.layers[4](enc_emb,state)

    # PLACE HOLDER FOR PREDECTED WORDS
    pred = []
    # PLACE HOLDER FOR STATES 
    input_state = [state_h,state_c]
    # CURRENT VECTOR TO BE PASSED TO DECODER 
    current_vec = tf.ones((1,1))
    
    for i in range(50): # FOR i UP TO 50 (MAX LENGTH)
        ## CONVERT THE CURRENT VECTOR SEQUENCE WORD TO EMBEDDINGS
        dec_emb  = model.layers[3](current_vec)
        ## PASSING EMBEDDED VECTOR TO DECODER LSTM LAYER
        dec_output,dec_state_h,dec_state_c = model.layers[5](dec_emb , input_state)
        # PASSING DECODER OUTPUT TO DENSE LAYER
        dense = model.layers[6](dec_output)

        # SELECTING INDEX OF MAXIMUM DENSE OUTPUT AS CURRENT VECTOR
        current_vec = np.argmax(dense ,axis = -1)
        # UPDATING THE INPUT STATES
        input_state = [dec_state_h,dec_state_c]

        # APPENDING THE ACTUAL TEXT TO "pred" VARIABLE
        pred.append(tk_out.index_word[current_vec[0][0]])
        ## IF THE CURRENT VECTOR IS "<end>" BREAK THE LOOP
        if tk_out.index_word[current_vec[0][0]]=="<end>":
            break
    ## RETURN THE JOINED STRING IN LIST "pred"
    return " ".join(pred)

In [26]:
# Prediction on Test Set
print("INPUT SENTENCE ===> ",df_test.enc_input.values[10])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[10],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[10])

INPUT SENTENCE ===>  इसी बीच अमरीकी विदेश मंत्री हिलेरीक्लिंटन ने कहा हैं कि विकीलीक्स को जानकारी देने वालों के विरुद्ध कड़ी कार्रवाई की जाएगी ।
PREDICTED SENTENCE ===>  इसी बीच अमरीकी विदेश मंत्री हिलेरीक्लिंटन ने कहा है कि विकीलीक्स को जानकारी देकर सुरक्षा के बदले की जानकारी होनी चाहिए । <end>
ACTUAL SENTENCE ===>  इसी बीच अमरीकी विदेश मंत्री हिलेरीक्लिंटन ने कहा है कि विकीलीक्स को जानकारी देने वालों के विरुद्ध कड़ी कार्रवाई की जाएगी ।  <end>


In [27]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[4])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[4],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[4])

INPUT SENTENCE ===>  वह एक गिलास शर्बत
PREDICTED SENTENCE ===>  उसने एक गिलास शर्बत <end>
ACTUAL SENTENCE ===>  उसने एक गिलास शर्बत <end>


In [28]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[20])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[20],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[20])

INPUT SENTENCE ===>  गुजरात और यूपी भी में हुए ऐसे मामले
PREDICTED SENTENCE ===>  गुजरात में किसे लेने की जल्द जानकारी मामले <end>
ACTUAL SENTENCE ===>  गुजरात और यूपी में भी हुए ऐसे मामले <end>


In [29]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[500])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[500],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[500])

INPUT SENTENCE ===>  अमेरिका भी इससे अलग नहीं हैं ।
PREDICTED SENTENCE ===>  अमेरिका भी इससे अलग नहीं है । <end>
ACTUAL SENTENCE ===>  अमेरिका भी इससे अलग नहीं है ।  <end>


In [34]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[900])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[900],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[900])

INPUT SENTENCE ===>  इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई हैं ।
PREDICTED SENTENCE ===>  इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई है । <end>
ACTUAL SENTENCE ===>  इसलिए वैज्ञानिकों ने लाइकोपीन आधारिक एक दवाई बनाई है ।  <end>


In [36]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[700])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[700],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[700])

INPUT SENTENCE ===>  जनपथ के एक दुकानदार नितिन कहते है कि अबतक उन्होंने किसी को भी पॉलिथीन चेकिंग करते नहीं देखा
PREDICTED SENTENCE ===>  जनपथ के एक दुकानदार नितिन कहते हैं कि अबतक उन्होंने किसी को भी पॉलिथीन चेकिंग करते नहीं देखा <end>
ACTUAL SENTENCE ===>  जनपथ के एक दुकानदार नितिन कहते हैं कि अबतक उन्होंने किसी को भी पॉलिथीन चेकिंग करते नहीं देखा <end>


In [45]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[550])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[550],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[550])

INPUT SENTENCE ===>  पांचवें दिन फौज के लोगों को फकीरी दी जाती थे ।
PREDICTED SENTENCE ===>  पांचवें दिन फौज के लोगों को फकीरी दी जाती थी । <end>
ACTUAL SENTENCE ===>  पांचवें दिन फौज के लोगों को फकीरी दी जाती थी ।  <end>


In [39]:
print(df_test.shape)

(1000, 3)


In [30]:
%%time
#Inference Time
predict(df_test.enc_input.values[50],model)

CPU times: user 209 ms, sys: 4.57 ms, total: 214 ms
Wall time: 180 ms


'नारों की आवाज मुखिया को ठीक उसी तरह खीच लाई जैसे बीन की धुन सांप को बाहर खीच लाती है । <end>'

In [33]:
#BELU SCore
import nltk.translate.bleu_score as bleu

In [56]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[550])
p = predict(df_test.enc_input.values[550],model)
a = df_test.dec_output.values[550]
print("PREDICTED SENTENCE ===> ",p)
print("ACTUAL SENTENCE ===> ",a)
# import nltk.translate.bleu_score as bleu
# bleu.sentence_bleu(a,p)

INPUT SENTENCE ===>  पांचवें दिन फौज के लोगों को फकीरी दी जाती थे ।
PREDICTED SENTENCE ===>  पांचवें दिन फौज के लोगों को फकीरी दी जाती थी । <end>
ACTUAL SENTENCE ===>  पांचवें दिन फौज के लोगों को फकीरी दी जाती थी ।  <end>


In [53]:
# VALIDATION BELU SCORE
import nltk.translate.bleu_score as bleu
BLEU_val_emb = []
test_data = df_val.loc[np.random.choice(df_val.index,size = 1000)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_val_emb.append(b)
    except:
        continue

1000it [02:08,  7.80it/s]


In [54]:
print("BELU Score = ",np.mean(BLEU_val_emb))

BELU Score =  0.31729669098268243


In [59]:
BLEU_val_emb = []
test_data = df_val.loc[np.random.choice(df_val.index,size = 1000)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_val_emb.append(b)
    except:
        continue
print("BELU Score = ",np.mean(BLEU_val_emb))

1000it [02:08,  7.80it/s]

BELU Score =  0.31729669098268243


In [57]:
#New test set (all together a different dataset)
print(test_data.shape)

(2000, 3)


In [60]:
df_test1= pd.read_csv("DATA/etoori_test.csv")
df_test1.columns = ["enc_input","dec_input"] 
df_test1["dec_output"] = df_test1.dec_input
print(df_test1.shape)
df_test1

(30000, 3)


,enc_input,dec_input,dec_output
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...
1,यह मन को काबू में करने वाली मुद्रा हैं इसीलिए ...,यह मन को काबू में करने वाली मुद्रा है इसीलिए इ...,यह मन को काबू में करने वाली मुद्रा है इसीलिए इ...
2,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ...,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ प...,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ प...
3,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...
4,उनकी वो वाली बात भी अनिश्चित रहती हैं ।,उनकी वो वाली बात भी अनिश्चित रहती है ।,उनकी वो वाली बात भी अनिश्चित रहती है ।
...,...,...,...
29995,पालिश घर्षक मशीनें करती हैंं ।,पालिश घर्षक मशीनें करती हैं ।,पालिश घर्षक मशीनें करती हैं ।
29996,पहले पाकिस्तान ने हमले में भारत का हाथ बताया औ...,पहले पाकिस्तान ने हमले में भारत का हाथ बताया औ...,पहले पाकिस्तान ने हमले में भारत का हाथ बताया औ...
29997,सारे का सारा दर्द उन्हों अप ह्रदय में ही समेट ...,सारे का सारा दर्द उन्होंने अपने ह्रदय में ही स...,सारे का सारा दर्द उन्होंने अपने ह्रदय में ही स...
29998,विल्सन ने लिखे पत्र में रिपब्लिकन सांसदों से भ...,विल्सन ने लिखे पत्र में रिपब्लिकन सांसदों से भ...,विल्सन ने लिखे पत्र में रिपब्लिकन सांसदों से भ...


In [62]:
print("INPUT SENTENCE ===> ",df_test1.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test1.enc_input.values[50],model))
print("ACTUAL SENTENCE ===> ",df_test1.dec_output.values[50])

INPUT SENTENCE ===>  घर में घुस के बाद भी सुकून नहीं ।
PREDICTED SENTENCE ===>  घर में घुसने के बाद भी आज्ञा दिया । <end>
ACTUAL SENTENCE ===>  घर में घुसने के बाद भी सुकून नहीं । 


In [65]:
print("INPUT SENTENCE ===> ",df_test1.enc_input.values[1000])
print("PREDICTED SENTENCE ===> ",predict(df_test1.enc_input.values[1000],model))
print("ACTUAL SENTENCE ===> ",df_test1.dec_output.values[1000])

INPUT SENTENCE ===>  जगजीत ने ग़ज़लों लोकप्रिय बनाने के लिए बहुत मेहनत की और हर विधा से उसकी प्रचार किया ।
PREDICTED SENTENCE ===>  जगजीत वाजपेयी ने कार्यशाला होने के लिए अच्छी और आरक्षित से हर क्षण को अपने लिए किया । <end>
ACTUAL SENTENCE ===>  जगजीत ने ग़ज़लों लोकप्रिय बनाने के लिए बहुत मेहनत की और हर विधा से उसका प्रचार किया । 


In [67]:
df_test1 = df_test1.head(1000)
df_test1["dec_input"]= "<start> " + df_test1["dec_input"]
df_test1["dec_output"] =  df_test1["dec_output"] + " <end>" 


In [68]:
BLEU_val_emb = []
test_data = df_test1
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_val_emb.append(b)
    except:
        continue
print("BELU Score = ",np.mean(BLEU_val_emb))

1000it [02:06,  7.93it/s]

BELU Score =  0.3104000846208022


In [71]:
print("INPUT SENTENCE ===> ",df_test1.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test1.enc_input.values[50],model))
print("ACTUAL SENTENCE ===> ",df_test1.dec_output.values[50])

INPUT SENTENCE ===>  घर में घुस के बाद भी सुकून नहीं ।
PREDICTED SENTENCE ===>  घर में घुसने के बाद भी आज्ञा दिया । <end>
ACTUAL SENTENCE ===>  घर में घुसने के बाद भी सुकून नहीं ।  <end>


In [73]:
df_test1= pd.read_csv("DATA/etoori_test.csv")
df_test1.columns = ["enc_input","dec_input"] 
df_test1["dec_output"] = df_test1.dec_input
print(df_test1.shape)


(30000, 3)


In [74]:
df_test1 = df_test1.head(10000)
df_test1["dec_input"]= "<start> " + df_test1["dec_input"]
df_test1["dec_output"] =  df_test1["dec_output"] + " <end>" 

In [75]:
BLEU_val_emb = []
test_data = df_test1
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_val_emb.append(b)
    except:
        continue
print("BELU Score = ",np.mean(BLEU_val_emb))

(10000, 3)


10000it [21:49,  7.64it/s]

BELU Score =  0.3049701667824825
